# Decoding Cellphone Signals with a Quantum Computer
This notebook demonstrates

## What is Coordinated Multipoint?

TBD: add description, papers, etc

# Modeling a Network

TBD: explain the King's graph-based network model

The first code cell imports the needed funcions
NOTE: The code imports from "helpers", a folder colocated with this Jupyter Notebook.

In [ ]:
# General Python packages
import matplotlib.pyplot as plt
import numpy as np

# Ocean software packages
import dimod
from dwave.samplers import SimulatedAnnealingSampler
from dwave.system import DWaveSampler, FixedEmbeddingComposite

# Functions of this repository
from helpers.draw import draw_loop_comparison, draw_network
from helpers.filters import apply_filter, apply_filters, compare_signals, create_filter, create_filters, time_filter_instantiation
from helpers.general import loop_comparisons
from helpers.network import configure_network, create_channels, print_network_stats, simulate_signals

# Enable graphics inline
%matplotlib inline 

## Create a Network Graph

TBD: explain the King's graph-based network model

NOTE: The code imports from "helpers", a folder colocated with this Jupyter Notebook.

In [ ]:
network, _ = configure_network(lattice_size=5)

print_network_stats(network)
draw_network(network)

The next code cell plots the network graph, showing transmitters in red and receivers in green. 

## Create Channels

Between the transmitters and receivers there are channels etc

In [ ]:
channels, channel_power = create_channels(network)
print(f"Channels are represented by a {channels.shape[0]}x{channels.shape[1]} matrix.")

# Decoding Transmissions: Classical Resources

TBD: explain linear filters

## Create Filters
Explain about filters

In [ ]:
filters = create_filters(channels)
print(f"Created filters: {list(filters.keys())}.")

## Simulate Transmissions


In [ ]:
y, transmitted_symbols = simulate_signals(channels, channel_power)                 
print(f"First 10 transmitted symbols: {transmitted_symbols.flatten()[:10]}. \nFirst 10 received symbols: {y.flatten()[:10]}.")

## Decode Received Signals

In [ ]:
v = apply_filters(y, filters)
compare_signals(v, transmitted_symbols)

# Scaling Up

PLACEHOLDER TEXT: The function below compares ..... Times greater than half a second are highlighted in red. 

In [ ]:
time_filter_instantiation(network_size=[5, 10, 15])

# Decoding Transmissions: Quantum
Some words here

In [ ]:
qpu = DWaveSampler(solver=dict(topology__type="pegasus"))

print(f"Selected {qpu.solver.name} with {len(qpu.nodelist)} qubits.")

## BQM Representation (Small-Town Problem)

### Create a Network, Filter, and Simulate Signals

In [ ]:
SNR = 10

network, embedding = configure_network(lattice_size=16, qpu=qpu)
print_network_stats(network)

channels, channel_power =  create_channels(network)

filter_mf = create_filter(channels, method='matched_filter')
y, transmitted_symbols = simulate_signals(channels, channel_power, SNRb=SNR)  

For comparison, decode this tranmission with the linear filter.

In [ ]:
v = apply_filter(filter_mf, y) 
compare_signals(v, transmitted_symbols)

### Create a BQM

In [ ]:
bqm = dimod.generators.mimo.spin_encoded_comp(network, 
                                              modulation = 'BPSK', 
                                              transmitted_symbols=transmitted_symbols, 
                                              F_distribution=('binary','real'), 
                                              F=channels,
                                              y=y)

print(f"BQM has {len(bqm)} variables with {len(bqm.quadratic)} quadratic interactions.")

### Decode Received Signals

Use the quantum computer to decode the tranmission.

In [ ]:
sampler = FixedEmbeddingComposite(qpu, embedding)

sampleset = sampler.sample(bqm, 
                           num_reads=30, 
                           annealing_time=200, 
                           chain_strength=-0.13*min(bqm.linear.values()),
                           label='Notebook - Coordinated Multipoint')

compare_signals(sampleset, transmitted_symbols)

## Big-City Problems

PLACEHOLDER TEXT: Problems become harder as a function of noise (SNR) and transmitter-to-receiver ratio ($\frac{Tx}{Rx}$).

The image below shows ...

<img src="_static/problem_hardness.png" width="800">

In [ ]:
results = loop_comparisons(runs=10, qpu=qpu)
medians = {filter: np.median(val) for filter, val in results.items()}

for solver in medians:
    print(f'\t * Median success rate for {solver}: {medians[solver]}.')

draw_loop_comparison(results)

### Extreme Enviroment

In [ ]:
network, embedding = configure_network(lattice_size=16, ratio=1.65, qpu=qpu)
print_network_stats(network)

SNR = 4.5

channels, channel_power =  create_channels(network)

filter_mf = create_filter(channels, method='matched_filter', snr_over_nt=SNR)
y, transmitted_symbols = simulate_signals(channels, channel_power, SNRb=SNR)

bqm = dimod.generators.mimo.spin_encoded_comp(network, 
                                              modulation = 'BPSK', 
                                              transmitted_symbols=transmitted_symbols, 
                                              F_distribution=('binary','real'), 
                                              F=channels,
                                              y=y)

In [ ]:
sampler = FixedEmbeddingComposite(qpu, embedding)

sampleset = sampler.sample(bqm, 
                           num_reads=30, 
                           annealing_time=200, 
                           chain_strength=-0.13*min(bqm.linear.values()),
                           label='Notebook - Coordinated Multipoint')

compare_signals(sampleset, transmitted_symbols)

In [ ]:
v = apply_filter(filter_mf, y) 
compare_signals(v, transmitted_symbols)

# Supplementary Technical Material

## Performance of Inverting Linear Filters (Success Rate and Time)

In [ ]:
# Graphic comparison goes here

In [ ]:
# TBD: move this code to helper file

results = {"QPU": []}		
lattice_sizes = [4, 8, 12, 16] 
for lattice_size in lattice_sizes:
    network, embedding = configure_network(lattice_size=16, ratio=1.5, qpu=qpu)
    print_network_stats(network)
    sampler = FixedEmbeddingComposite(qpu, embedding)

    for run in range(3):
        channels, channel_power = create_channels(network)
    
        filters = create_filters(channels)
    
        y, transmitted_symbols = simulate_signals(channels, channel_power, SNRb=SNR)   
        bqm = dimod.generators.mimo.spin_encoded_comp(network, 
                                                  modulation = 'BPSK', 
                                                  transmitted_symbols=transmitted_symbols, 
                                                  F_distribution=('binary','real'), 
                                                  F=channels,
                                                  y=y)
        
        sampleset = sampler.sample(bqm, num_reads=30, 
                               annealing_time=200, 
                               chain_strength=-0.13*min(bqm.linear.values()), 
                               label='Notebook - Coordinated Multipoint')
    
        results["QPU"].append(round(100*sum(np.array(list(sampleset.first.sample.values())) == transmitted_symbols.flatten())/len(transmitted_symbols)))
        
        v = apply_filters(y, filters)
        compare_signals(v, transmitted_symbols)
        #results["MF"].append(round(100*sum(v.flatten() == transmitted_symbols.flatten())/len(transmitted_symbols)))
    
 
#stats = {key: np.median(val) for key, val in results.items()}


In [ ]:
results

## Comparison to Simulated Annealing

PLACEHOLDER TEXT: Like linear filters that require matrix inversion, simulated annealing is also computationally demanding, requiring high power consumption. However, simulated annealing can produce high-quality results on the same order as the QPU.  

In [ ]:
# TBD: move this code to helper file

import numpy as np
import networkx as nx

network, embedding = configure_network(lattice_size=16, ratio=1.5, qpu=qpu)
print_network_stats(network)

sa_sampler = SimulatedAnnealingSampler()
qpu_sampler = FixedEmbeddingComposite(qpu, embedding)

results = {"QPU": [], "SA": []}
for run in range(10):
    transmitted_symbols = np.random.choice([1, -1], size=[sum(nx.get_node_attributes(network, "num_transmitters").values()), 1]) 

    bqm = dimod.generators.mimo.spin_encoded_comp(network, 
                                              modulation = 'BPSK', 
                                              transmitted_symbols=transmitted_symbols, 
                                              F_distribution=('binary','real'), 
                                              SNRb=5)

    sampleset_sa = sa_sampler.sample(bqm, num_reads=1, num_sweeps=150)
    sampleset_qpu = qpu_sampler.sample(bqm, num_reads=30, 
                           annealing_time=200, 
                           chain_strength=-0.13*min(bqm.linear.values()), 
                           label='Notebook - Coordinated Multipoint')

    results["QPU"].append(round(100*sum(np.array(list(sampleset_qpu.first.sample.values())) == transmitted_symbols.flatten())/len(transmitted_symbols)))
    results["SA"].append(round(100*sum(np.array(list(sampleset_sa.first.sample.values())) == transmitted_symbols.flatten())/len(transmitted_symbols)))


In [ ]:
# TBD: move this code to helper file

fig, ax = plt.subplots(nrows=1, ncols=1)
ax.plot(results["QPU"], "b*-", label=f"QPU", markersize=5)
ax.plot(results["SA"], "g^-", label=f"Simulated Annealing", markersize=5)
ax.set(xlabel="Seed", ylabel="Success Rate [%]")
plt.suptitle("P16, Tx/Rx=1.5, Anneal Time 200us")
ax.legend()

## Minor Embedding

The graphic below shows how the network graph is minor-embedded into the QPU working graph for a network of lattice size 4. 

Colored dots are qubits: note that pairs of identically-colored dots, such as the blue, green, and red pairs on the top row, represent a single node of the logical network. 

<img src="_static/embedding_lattice4.png">
